In [12]:
#| default_exp core

# Mistinguette's source 

## Setup

In [48]:
#|export
import os
from collections import abc
try: from IPython import display
except: display=None
from fastcore.utils import *
from rich import print

from mistralai import Mistral
from mistralai.models import ChatCompletionChoice, ChatCompletionResponse

In [49]:
MISTRAL_API_KEY = os.environ.get("MISTRAL_API_KEY")

In [15]:
#| exports
model_types = {
    # Premier models
    'codestral-2501': 'codestral-latest', # code generation model
    'mistral-large-2411': 'mistral-large-latest', # top-tier reasoning model for high-complexity tasks
    'pixtral-large-2411': 'pixtral-large-latest', # frontier-class multimodal model
    'mistral-saba-2502': 'mistral-saba-latest', # model for languages from the Middle East and South Asia
    'ministral-3b-2410': 'ministral-3b-latest', # edge model
    'ministral-8b-2410': 'ministral-8b-latest', # edge model with high performance/price ratio
    'mistral-embed-2312': 'mistral-embed', # embedding model
    'mistral-moderation-2411': 'mistral-moderation-latest', # moderation service to detect harmful text content
    'mistral-ocr-2503': 'mistral-ocr-latest', # OCR model to extract interleaved text and images
    
    # Free models (with weight availability)
    'mistral-small-2503': 'mistral-small-latest', # small model with image understanding capabilities
    
    # Research models
    'open-mistral-nemo-2407': 'open-mistral-nemo', # multilingual open source model
}

all_models = list(model_types)

In [16]:
#| export
models = all_models

In [17]:
model = models[1]; model

'mistral-large-2411'

## Mistral SDK

In [18]:
cli = Mistral(api_key=MISTRAL_API_KEY)

This is what Mistral's SDK provides for interacting with Python. To use it, pass it a list of *messages*, with *content* and a *role*. The roles should alternate between *user* and *assistant*.

In [19]:
# system, user, assistant, tool

In [20]:
m = {'role': 'user', 'content': "I'm Franck"}
r = cli.chat.complete(messages = [m], model = model)
r

ChatCompletionResponse(id='af69417214854aa4900cbf4881fdaa3e', object='chat.completion', model='mistral-large-2411', usage=UsageInfo(prompt_tokens=8, completion_tokens=27, total_tokens=35), created=1742682831, choices=[ChatCompletionChoice(index=0, message=AssistantMessage(content='Hello Franck! Nice to meet you. How are you today? Is there something specific you would like to talk about or do?', tool_calls=None, prefix=False, role='assistant'), finish_reason='stop')])

In [26]:
print(r)

ChatCompletionResponse(
    id='af69417214854aa4900cbf4881fdaa3e',
    object='chat.completion',
    model='mistral-large-2411',
    usage=UsageInfo(prompt_tokens=8, completion_tokens=27, total_tokens=35),
    created=1742682831,
    choices=[
        ChatCompletionChoice(
            index=0,
            message=AssistantMessage(
                content='Hello Franck! Nice to meet you. How are you today? Is there something specific you would 
like to talk about or do?',
                tool_calls=None,
                prefix=False,
                role='assistant'
            ),
            finish_reason='stop'
        )
    ]
)

In [10]:
m = [
    {'role': 'system', 'content': "You are a helpful assistant full of irony"},
    {'role': 'user', 'content': "I'm Franck"}]
r = cli.chat.complete(messages = m, model = model)


In [11]:
[o for o in dir(r) if not o.startswith('_')]

['choices',
 'construct',
 'copy',
 'created',
 'dict',
 'from_orm',
 'id',
 'json',
 'model',
 'model_computed_fields',
 'model_config',
 'model_construct',
 'model_copy',
 'model_dump',
 'model_dump_json',
 'model_extra',
 'model_fields',
 'model_fields_set',
 'model_json_schema',
 'model_parametrized_name',
 'model_post_init',
 'model_rebuild',
 'model_validate',
 'model_validate_json',
 'model_validate_strings',
 'object',
 'parse_file',
 'parse_obj',
 'parse_raw',
 'schema',
 'schema_json',
 'update_forward_refs',
 'usage',
 'validate']

In [36]:
r.model_fields_set

{'choices', 'created', 'id', 'model', 'object', 'usage'}

In [40]:
r.choices[0].model_fields_set

{'finish_reason', 'index', 'message'}

In [44]:
r.choices[0].message.model_fields_set

{'content', 'role', 'tool_calls'}

In [47]:
r.choices[0].message.content

"Well, Franck, it's a pleasure to meet you. I must say, I've always been a fan of the name. It's strong, it's classic, it's... frankly, it's fantastic. You've set a high bar for yourself, Franck. Let's hope you can live up to the grandeur of your name. So, how can I help you today, oh Franck the Magnificent?"

In [48]:
r.choices[0].message.role

'assistant'

In [24]:
r.choices[0].message.content

'Hello Franck! Nice to meet you. How are you today? Is there something you would like help with?'

In [57]:
# Notes:
#  - assistant message with prefix true, should be last message
#  - assistant message with prefix false cannot be last.

In [58]:
m = [
    {'role': 'system', 'content': "You are a helpful assistant full of irony"},
    {'role': 'user', 'content': "I'm Franck"},
    {'role': 'assistant', 'content': "Well, Franck, it's a pleasure to meet you. I must say, I've always been a fan of the name. It's strong, it's classic, it's... frankly, it's fantastic. You've set a high bar for yourself, Franck. Let's hope you can live up to the grandeur of your name. So, how can I help you today, oh Franck the Magnificent?"
},
    {'role': 'user', 'content': "Hum I don't like your irony"}
    ]
r = cli.chat.complete(messages = m, model = model)


In [59]:
r.choices[0].message.content

"I apologize if my previous response didn't resonate with you. I'll make sure to keep my irony in check. So, how can I assist you today, Franck? Let's start over. I'm here to help."

In [60]:
r.choices

[ChatCompletionChoice(index=0, message=AssistantMessage(content="I apologize if my previous response didn't resonate with you. I'll make sure to keep my irony in check. So, how can I assist you today, Franck? Let's start over. I'm here to help.", tool_calls=None, prefix=False, role='assistant'), finish_reason='stop')]

### Formatting output

In [10]:
r.choices[0]

ChatCompletionChoice(index=0, message=AssistantMessage(content="Hello Franck! Nice to meet you. How are you doing today? Let's have a friendly conversation. How about I share an interesting fact or a light joke to start? Here it goes:\n\nInteresting fact: Did you know that a day on Venus is longer than a year on Venus? It takes Venus about 243 Earth days to rotate once on its axis, but it only takes around 225 Earth days for Venus to orbit the Sun.\n\nOr, if you prefer a light joke:\n\nWhat do you call fake spaghetti?\nAn impasta!", tool_calls=None, prefix=False, role='assistant'), finish_reason='stop')

In [61]:
r

ChatCompletionResponse(id='3a264de440dd44d79721da1a831fc0bd', object='chat.completion', model='mistral-large-2411', usage=UsageInfo(prompt_tokens=128, completion_tokens=53, total_tokens=181), created=1742565189, choices=[ChatCompletionChoice(index=0, message=AssistantMessage(content="I apologize if my previous response didn't resonate with you. I'll make sure to keep my irony in check. So, how can I assist you today, Franck? Let's start over. I'm here to help.", tool_calls=None, prefix=False, role='assistant'), finish_reason='stop')])

In [ ]:
# For Anthropic
# Message(id='msg_01V27cEWaHGprN6nEdaJYZvF', content=[TextBlock(citations=None, text="Hello, Jeremy! It's nice to meet you. How are you doing today? Is there something I can help you with or would you like to chat about something specific?", type='text')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=10, output_tokens=38))

In [11]:
r.choices[0].message.content

"Hello Franck! Nice to meet you. How are you doing today? Let's have a friendly conversation. How about I share an interesting fact or a light joke to start? Here it goes:\n\nInteresting fact: Did you know that a day on Venus is longer than a year on Venus? It takes Venus about 243 Earth days to rotate once on its axis, but it only takes around 225 Earth days for Venus to orbit the Sun.\n\nOr, if you prefer a light joke:\n\nWhat do you call fake spaghetti?\nAn impasta!"

In [35]:
#| exports
def find_choice(r:abc.Mapping, # The message to look in
                chc_type:type=ChatCompletionChoice  # The type of choice to find
              ):
    "Find the first choice of type `chc_type` in `r.choices`."
    return first(o for o in r.choices if isinstance(o,chc_type))
     

In [36]:
find_choice(r)

ChatCompletionChoice(index=0, message=AssistantMessage(content='Hello Franck! Nice to meet you. How are you today? Is there something specific you would like to talk about or do?', tool_calls=None, prefix=False, role='assistant'), finish_reason='stop')

In [39]:
def contents(r):
    "Helper to get the contents from Mistral response `r`."
    chc = find_choice(r)
    if not chc and r.choices: chc = r.choices[0]
    msg = chc.message
    return msg.content.strip() if hasattr(msg,'content') else str(msg)

In [40]:
contents(r)

'Hello Franck! Nice to meet you. How are you today? Is there something specific you would like to talk about or do?'

In [50]:
#| exports
@patch
def _repr_markdown_(self:(ChatCompletionResponse)):
    det = '\n- '.join(f'{k}: `{v}`' for k,v in self.model_dump().items())
    cts = re.sub(r'\$', '&#36;', contents(self))  # escape `$` for jupyter latex
    return f"""{cts}

<details>

- {det}

</details>"""

In [51]:
r

Hello Franck! Nice to meet you. How are you today? Is there something specific you would like to talk about or do?

<details>

- id: `af69417214854aa4900cbf4881fdaa3e`
- object: `chat.completion`
- model: `mistral-large-2411`
- usage: `{'prompt_tokens': 8, 'completion_tokens': 27, 'total_tokens': 35}`
- created: `1742682831`
- choices: `[{'index': 0, 'message': {'content': 'Hello Franck! Nice to meet you. How are you today? Is there something specific you would like to talk about or do?', 'tool_calls': None, 'prefix': False, 'role': 'assistant'}, 'finish_reason': 'stop'}]`

</details>

In [52]:
r.usage

UsageInfo(prompt_tokens=8, completion_tokens=27, total_tokens=35)